*The following is a very naive implementation of a hidden markov model for Part of Speech tagging*<br/>
*Several improvements are possible: word preprocessing, handling OOV words etc.*<br/>
*The current state of the art transformers do POS tagging with an accuracy of ~98%*<br/>
*I was able to acheive a decent matching (90% on 25% held out test corpus) as well with this statistical model*<br/>
*However this was due to the "universal" taglist which is not as broad (12 labels) instead of the (46 labels) in the original dataset*

In [1]:
import nltk

In [2]:
dataset = nltk.corpus.treebank

In [4]:
tagged_sents = list(dataset.tagged_sents(tagset="universal"))
# tagged_sents = list(dataset.tagged_sents())

In [5]:
tagged_sents[:5]

[[('Pierre', 'NOUN'),
  ('Vinken', 'NOUN'),
  (',', '.'),
  ('61', 'NUM'),
  ('years', 'NOUN'),
  ('old', 'ADJ'),
  (',', '.'),
  ('will', 'VERB'),
  ('join', 'VERB'),
  ('the', 'DET'),
  ('board', 'NOUN'),
  ('as', 'ADP'),
  ('a', 'DET'),
  ('nonexecutive', 'ADJ'),
  ('director', 'NOUN'),
  ('Nov.', 'NOUN'),
  ('29', 'NUM'),
  ('.', '.')],
 [('Mr.', 'NOUN'),
  ('Vinken', 'NOUN'),
  ('is', 'VERB'),
  ('chairman', 'NOUN'),
  ('of', 'ADP'),
  ('Elsevier', 'NOUN'),
  ('N.V.', 'NOUN'),
  (',', '.'),
  ('the', 'DET'),
  ('Dutch', 'NOUN'),
  ('publishing', 'VERB'),
  ('group', 'NOUN'),
  ('.', '.')],
 [('Rudolph', 'NOUN'),
  ('Agnew', 'NOUN'),
  (',', '.'),
  ('55', 'NUM'),
  ('years', 'NOUN'),
  ('old', 'ADJ'),
  ('and', 'CONJ'),
  ('former', 'ADJ'),
  ('chairman', 'NOUN'),
  ('of', 'ADP'),
  ('Consolidated', 'NOUN'),
  ('Gold', 'NOUN'),
  ('Fields', 'NOUN'),
  ('PLC', 'NOUN'),
  (',', '.'),
  ('was', 'VERB'),
  ('named', 'VERB'),
  ('*-1', 'X'),
  ('a', 'DET'),
  ('nonexecutive', 'ADJ'),
 

In [6]:
len(tagged_sents)

3914

In [7]:
n = len(tagged_sents)
train_data = tagged_sents[:int(0.75*n)]
test_data = tagged_sents[int(0.75*n):]
print(len(train_data), len(test_data))

2935 979


In [36]:
word_label_pair_train = [i for sent in tagged_sents for i in sent]
print(len(word_label_pair_train))

100676


In [37]:
vocab_train = list(set([pair[0] for pair in word_label_pair_train]))
tags = list(set([pair[1] for pair in word_label_pair_train]))
print(f"Number of tokens: {len(vocab_train)}, and the number of tags are: {len(tags)}")

Number of tokens: 12408, and the number of tags are: 12


In [38]:
print(tags)

['VERB', 'ADP', 'PRON', 'X', 'PRT', 'NUM', '.', 'ADJ', 'DET', 'CONJ', 'ADV', 'NOUN']


In [39]:
word_to_index = {word: i for i, word in enumerate(vocab_train)}
index_to_word = {i: word for i, word in enumerate(vocab_train)}
tags_to_index = {tag: i for i, tag in enumerate(tags)}
index_to_tags = {i: tag for i, tag in enumerate(tags)}

print(word_to_index["will"], index_to_word[word_to_index["will"]])

4018 will


**The tags are as follows: <br/>
[link](https://www.eecis.udel.edu/~vijay/cis889/ie/pos-set.pdf)**

**Emmission probabilities**: The P(w|t) = C(w, t)/C(t)

In [40]:
import numpy as np

tags_count = len(tags)
vocab_size = len(vocab_train)
emmission_counts = np.ones((tags_count, vocab_size)) # add-one smoothing

In [41]:
def emmision_count(word, tag, data=word_label_pair_train):
    """
    Parameters:
        word: (string),
        tag: (string),
        data: (list of tuples {(word, tag)})
    Returns:
        (count_word_as_tag, count_tag): (int, int)
    """

    tag_appearance = [tup for tup in data if tup[1] == tag]
    count_tag = len(tag_appearance)
    word_as_tag = [tup for tup in tag_appearance if tup[0] == word]
    count_word_as_tag = len(word_as_tag)
    return count_word_as_tag, count_tag

In [ ]:
# count_will_as_MD, count_MD = emmision_count("will", "MD")
# print(f"Count of word 'will' as tag 'MD' is {count_will_as_MD} and count of tag 'MD' is {count_MD} | percentage: {count_will_as_MD/count_MD * 100}")

In [42]:
# updating the emmission_counts matrix
for i, tag in enumerate(tags):
    for j, word in enumerate(vocab_train):
        emmission_counts[i, j] += emmision_count(word, tag)[0]
    print(f"{index_to_tags[i]} is done")

VERB is done
ADP is done
PRON is done
X is done
PRT is done
NUM is done
. is done
ADJ is done
DET is done
CONJ is done
ADV is done
NOUN is done


In [17]:
# emmission_counts[tags_to_index["MD"], word_to_index["will"]]

In [43]:
# print(index_to_word[np.argmax(emmission_counts[tags_to_index["MD"]])], 
#       index_to_word[np.argmax(emmission_counts[tags_to_index["DT"]])],
#       index_to_word[np.argmax(emmission_counts[tags_to_index["JJ"]])])

print(index_to_word[np.argmax(emmission_counts[tags_to_index["VERB"]])], 
      index_to_word[np.argmax(emmission_counts[tags_to_index["ADJ"]])],
      index_to_word[np.argmax(emmission_counts[tags_to_index["PRON"]])])

is new it


In [44]:
# to obtain the probabilities 
count_tags = np.zeros(tags_count)
for i, tag in enumerate(tags):
    count_tags[i] = emmision_count("", tag)[1]

In [45]:
count_tags = count_tags.reshape(-1, 1)

emmission_probs = emmission_counts/count_tags

In [46]:
# emmission_counts[tags_to_index["MD"], word_to_index["will"]], emmission_probs[tags_to_index["MD"], word_to_index["will"]]
emmission_counts[tags_to_index["VERB"], word_to_index["will"]], emmission_probs[tags_to_index["VERB"], word_to_index["will"]]

(281.0, 0.020716602772043645)

**Transition probabilities** P(t2|t1) = C(t1, t2)/C(t1): probability of a tag given a previous tag

In [47]:
def get_transition_proba(tag2, tag1, data=word_label_pair_train):
    """
    Parameters:
        tag1: (string),
        tag2: (string),
    Returns:
        percentage: (float) P(t2|t1)*100
    """
    count_tag1 = 0
    count_tag1_tag2 = 0
    tag_list = [tup[1] for tup in data]
    for t1, t2 in zip(tag_list, tag_list[1:]):
        if t1 == tag1:
            count_tag1 += 1
            if t2 == tag2:
                count_tag1_tag2 += 1
    return count_tag1_tag2/count_tag1 * 100

In [48]:
# print("Transition probability of VB to MD is: ", get_transition_proba("VB", "MD")) # prob of a verb following a modal verb
print("Transition probability of VERB to ADV is: ", get_transition_proba("VERB", "ADV"))

Transition probability of VERB to ADV is:  34.46862188584043


In [49]:
# build the transition matrix
transition_probs = np.zeros((tags_count, tags_count)) # transition_probs[i, j] = P(t_j|t_i)
for i in range(len(tags)):
    for j in range(len(tags)):
        transition_probs[i, j] = get_transition_proba(tags[j], tags[i]) / 100

In [50]:
# transition_probs[tags_to_index["MD"], tags_to_index["VB"]]
transition_probs[tags_to_index["ADV"], tags_to_index["VERB"]]

0.3446862188584043

![viterbi](./images/viterbi.png)

**VITERBI ALGORITHM**

In [51]:
initial_state_probs = [] # initial_state_probs[i] = prob that a sentence starts with tag i
for tag in tags:
    initial_state_probs.append(get_transition_proba(tag, ".")/100)

In [52]:
print(np.argmax(initial_state_probs), tags_to_index["NOUN"])

11 11


In [53]:
def Viterbi(sentence):
    """
        Parameters:
            sentence: (list of strings)
        Returns:
            tags: (list of strings)
    """
    viterbi = np.zeros((tags_count, len(sentence)))
    backpointer = np.zeros((tags_count, len(sentence)), dtype=int)
    for i in range(tags_count):
        viterbi[i, 0] = initial_state_probs[i] * emmission_probs[i, word_to_index[sentence[0]]]
        backpointer[i, 0] = 0
    for t in range(1, len(sentence)):
        for s in range(tags_count):
            # error if word is not in train vocab
            if sentence[t] not in vocab_train:
                viterbi[s, t] = np.max(viterbi[:, t-1] * transition_probs[:, s])
                backpointer[s, t] = np.argmax(viterbi[:, t-1] * transition_probs[:, s])
                continue
            viterbi[s, t] = np.max(viterbi[:, t-1] * transition_probs[:, s]) * emmission_probs[s, word_to_index[sentence[t]]]
            backpointer[s, t] = np.argmax(viterbi[:, t-1] * transition_probs[:, s])
    best_path_prob = np.max(viterbi[:, -1])
    best_path_pointer = np.argmax(viterbi[:, -1])
    best_path = [best_path_pointer]
    for i in range(len(sentence)-1, 0, -1):
        best_path_pointer = backpointer[best_path_pointer, i]
        best_path.append(best_path_pointer)
    best_path.reverse()
    return best_path, best_path_prob

In [54]:
# let us test it on the first train sentence
sentence = tagged_sents[78]
sentence_words = [tup[0] for tup in sentence]
sentence_tags = [tup[1] for tup in sentence]

best_path, _ = Viterbi(sentence_words)

print("The sentence is: ", sentence_words)
print(f"act.:{sentence_tags}")
print(f"pre.:{[index_to_tags[i] for i in best_path]}")
print(f"The percentage match is: {sum([1 if i==j else 0 for i, j in zip(sentence_tags, [index_to_tags[i] for i in best_path])])/len(sentence_tags) * 100}")

The sentence is:  ['The', 'governor', 'could', "n't", 'make', 'it', ',', 'so', 'the', 'lieutenant', 'governor', 'welcomed', 'the', 'special', 'guests', '.']
act.:['DET', 'NOUN', 'VERB', 'ADV', 'VERB', 'PRON', '.', 'ADP', 'DET', 'NOUN', 'NOUN', 'VERB', 'DET', 'ADJ', 'NOUN', '.']
pre.:['DET', 'NOUN', 'VERB', 'ADV', 'VERB', 'PRON', '.', 'ADV', 'DET', 'ADJ', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', '.']
The percentage match is: 81.25


In [55]:
# testing 
import random
rnd = [random.randint(0, len(tagged_sents)) for i in range(5)]
for i in rnd:
    sentence = tagged_sents[i]
    sentence_words = [tup[0] for tup in sentence]
    sentence_tags = [tup[1] for tup in sentence]
    best_path, _ = Viterbi(sentence_words)
    print("The sentence is: ", sentence_words)
    print(f"act.:{sentence_tags}")
    print(f"pre.:{[index_to_tags[i] for i in best_path]}")
    print(f"The percentage match is: {sum([1 if i==j else 0 for i, j in zip(sentence_tags, [index_to_tags[i] for i in best_path])])/len(sentence_tags) * 100}")
    print("\n")

The sentence is:  ['The', 'reduced', 'dividend', 'is', 'payable', 'Jan.', '2', 'to', 'stock', 'of', 'record', 'Dec.', '15', '.']
act.:['DET', 'VERB', 'NOUN', 'VERB', 'ADJ', 'NOUN', 'NUM', 'PRT', 'NOUN', 'ADP', 'NOUN', 'NOUN', 'NUM', '.']
pre.:['DET', 'ADJ', 'NOUN', 'VERB', 'X', 'PRT', 'NUM', 'PRT', 'NOUN', 'ADP', 'NOUN', '.', 'NUM', '.']
The percentage match is: 71.42857142857143


The sentence is:  ['Some', 'Democrats', ',', 'led', '*', 'by', 'Rep.', 'Jack', 'Brooks', '-LRB-', 'D.', ',', 'Texas', '-RRB-', ',', 'unsuccessfully', 'opposed', 'the', 'measure', 'because', 'they', 'fear', 'that', 'the', 'fees', 'may', 'not', 'fully', 'make', 'up', 'for', 'the', 'budget', 'cuts', '.']
act.:['ADV', 'NOUN', '.', 'VERB', 'X', 'ADP', 'NOUN', 'NOUN', 'NOUN', '.', 'NOUN', '.', 'NOUN', '.', '.', 'ADV', 'VERB', 'DET', 'NOUN', 'ADP', 'PRON', 'VERB', 'ADP', 'DET', 'NOUN', 'VERB', 'ADV', 'ADV', 'VERB', 'PRT', 'ADP', 'DET', 'NOUN', 'NOUN', '.']
pre.:['DET', 'NOUN', '.', 'VERB', 'X', 'ADP', 'NOUN', 'NOUN

In [56]:
total = 0
correct = 0
for sent in test_data:
  sent_words = [pair[0] for pair in sent]
  sent_tags = [pair[1] for pair in sent]  
  best_path, _ = Viterbi(sent_words)
  correct += sum([1 if i==j else 0 for i, j in zip(sent_tags, [index_to_tags[k] for k in best_path])])
  total += len(sent_tags)

print(f"Test accuracy: {(correct/total)*100}")

Test accuracy: 90.97300337457817
